In [ ]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import re
from datetime import datetime

In [ ]:
#Loading dataset

In [ ]:
df=pd.read_csv('All_Errors_Corrected.csv',parse_dates=['Standardized_Date'])
# df

In [ ]:
def categorize_remarks(data):
    """
    Categorizes the remarks column based on specific conditions:
    1. 'Power Failure at Intake' if:
       - Contains 'intake' (case-insensitive)
       - Contains 'power' (case-insensitive)
       - Does not contain 'wtp' (case-insensitive)
    2. 'Power Failure at WTP' if:
       - Does not contain 'intake'
       - Contains 'power' (case-insensitive)
       - Contains 'wtp' (case-insensitive)
    """
    def categorize(row):
        remarks = row['REMARKS']
        raw_water = row['RAW WATER FLOW IN m3/h']
        clear_water_pumping = row['CLEAR WATER PUMPING FLOW m3/h']
        
        if pd.notna(remarks):
            remarks_lower = remarks.lower()
            power_failure = 'power' in remarks_lower or 'failure' in remarks_lower
            has_intake = 'intake' in remarks_lower
            has_wtp = 'wtp' in remarks_lower
            
            if (has_intake and power_failure and not has_wtp) or (power_failure and not has_wtp and (raw_water == 0.0 or pd.isna(raw_water)) and not pd.isna(clear_water_pumping) and clear_water_pumping != 0.0):
                return 'Power Failure at Intake'
            elif (not has_intake and power_failure and has_wtp) or (power_failure and not has_intake and not pd.isna(raw_water) and raw_water !=0.0 and (clear_water_pumping == 0.0 or pd.isna(clear_water_pumping))):
                return 'Power Failure at WTP'
            elif (has_intake and power_failure and has_wtp) or (not has_intake and not has_wtp and power_failure and (raw_water == 0.0 or pd.isna(raw_water)) and (clear_water_pumping == 0.0 or pd.isna(clear_water_pumping))):
                return 'Power Failure at intake and WTP'
            elif ('cleaning' in remarks_lower or 'wash' in remarks_lower) and has_intake:
                return 'intake cleaning'
            elif 'maint' in remarks_lower:
                return 'maintanance'
#         return 'Other'
 
    # Apply the categorization function to each row
    data['remarks category'] = data.apply(categorize, axis=1)
    return data

In [ ]:
df=categorize_remarks(df)
df

In [ ]:
df[df['REMARKS'] == 'power failed @9.45 to 2.25']

In [ ]:
df[df['remarks category'] == 'Other']['REMARKS'].unique()

In [ ]:
# Function to fill NaN values in 'remarks category' based on the conditions
def fill_remarks_category(df):
    previous_remark = None

    for i in range(len(df)):
        current_remark = df.loc[i, 'remarks category']
        raw_water_flow = df.loc[i, 'RAW WATER FLOW IN m3/h']
        clear_water_pumping_flow = df.loc[i, 'CLEAR WATER PUMPING FLOW m3/h']

        # Check if 'remarks category' is NaN and if the previous row had a non-NaN remark
        if pd.isna(current_remark):
            # Check if raw water and treated water columns are 0 or NaN
            if ((raw_water_flow == 0 or pd.isna(raw_water_flow)) and (clear_water_pumping_flow == 0 or pd.isna(clear_water_pumping_flow))) or ((raw_water_flow == 0) and (clear_water_pumping_flow != 0)) or ((raw_water_flow != 0) and (clear_water_pumping_flow == 0)):
                # Fill the current NaN remark with the previous non-NaN remark
                df.loc[i, 'remarks category'] = previous_remark
            else:
                previous_remark = current_remark
                
        else:
            # Update the previous_remark if the current remark is not NaN
            previous_remark = current_remark

    return df

# Apply the function to the DataFrame
df_filled = fill_remarks_category(df)